# Setup and Load dataset



In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1EvuYBN9lIK8llpbUciP5jIOYgh0dpFHB

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EvuYBN9lIK8llpbUciP5jIOYgh0dpFHB
To: /content/bengali.zip
100% 13.5M/13.5M [00:00<00:00, 71.0MB/s]


In [ ]:
!unzip -o "bengali.zip"  -d  "/content"

Archive:  bengali.zip
  inflating: /content/train.csv      
  inflating: /content/valid.csv      


## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="train.csv"

###### Run below cell

In [ ]:
content_df  = pd.read_csv( file)

In [ ]:
content_df.head(10)

,title,article,label
0,এই অভিনেতার ‘প্রস্তাব’ মানলে ভারতীয় সিনেমার খো...,স্রেফ দু’টি টুইটেই সোশ্যাল মিডিয়ায় তোলপাড় ফে...,entertainment
1,"সচিন সাংসদ হয়েছেন, তাই চিন্তা কম জেটলির। কেন?",একবার রেল বাজেট। একবার সাধারণ বাজেট। খবরের শি...,national
2,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ...,sports
3,"দুষ্কৃতীদের নজরে এটিএম, হাল হকিকত্ ঘুরে দেখলেন...",কলকাতায় দুষ্কৃতীদের নজরে এখন বিভিন্ন ব্যাঙ্কের...,kolkata
4,"সৌজন্যের বালাই নেই, দিনভর আকচা আকচি কং-বিজেপি’র",সারা দিন আকচা আকচিতেই কাটিয়ে দিল দেশের দুই বৃহ...,national
5,নোট বদলের ধাক্কায় কেমন হল বিয়ে? জানুন,বিরিয়নিটা আর হোল না। ইচ্ছে ছিল বিয়েতে বিরায়ান...,kolkata
6,সিপিএমের মুখিয়া,এস কে এম কি আছেন?\nএ খোঁজ যে রাজ্য সিপিএমের ম...,state
7,গড়িয়ায় দফায় দফায় গণধর্ষণের শিকার অষ্টম শ্রেণির...,মহানগরের বুকে এবার দফায় দফায় গণধর্ষণের শিকার...,kolkata
8,বামেদের ধর্মঘটকে সমর্থন! নবান্ন অভিযানে বামেরা...,"আগামী বুধবার, ২ সেপ্টেম্বর বামেদের ডাকা বনধকে...",kolkata
9,আকাশবাড়ির হাতছানি,"পেরুতে গিয়ে যদি হোটেলের ঘরে বসে সময় কাটান, তা...",international


## Details about dataset

In [ ]:
itemid="title"

In [ ]:
features=['title','article']

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
content_df['clean'] = content_df['NewTag']


In [ ]:
content_df['clean'] =content_df['clean'].astype(str)

# Elastic search set up

In [ ]:
%%capture
!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [ ]:
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers


In [ ]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Creating daemon instance of elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [ ]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon      1226     857 99 06:55 ?        00:00:32 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.io.tmpdir=/tmp/elasticsearch-7620800406798313423 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Djava.locale.providers=COMPAT -Dio.netty.allocator.type=unpooled -Des.path.home=/content/elasticsearch-7.0.0 -Des.path.conf=/content/elasticsearch-7.0.0/config -Des.distribution.flavor=default 

In [ ]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

{
  "name" : "4e6ba3259808",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "QoYrwjjNQzOab0BfaAL7gA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

True

# Search BM25

In [ ]:
# Define settings & mappings of Elasticsearch index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0,    
    "analysis": {
      "filter": {
        "bengali_stop": {
          "type":       "stop",
          "stopwords":  "_bengali_" 
        },
        "bengali_keywords": {
          "type":       "keyword_marker",
          "keywords":   ["উদাহরণ"] 
        },
        "bengali_stemmer": {
          "type":       "stemmer",
          "language":   "bengali"
        }
      },
      "analyzer": {
        "rebuilt_bengali": {
          "tokenizer":  "standard",
          "filter": [
            "lowercase",
            "decimal_digit",
            "bengali_keywords",
            "indic_normalization",
            "bengali_normalization",
            "bengali_stop",
            "bengali_stemmer"
          ]
        }
      }
    }
  }
    ,
    "mappings":{
        "properties":{
            "id":{
                "type":"text"
            },
            "longtext":{
                "type":"text"
            }
        }
    }
}

In [ ]:
indexname='new_index1'
index_type='_doc'

In [ ]:

index = es.indices.create(index=indexname, ignore=[400,404], body=Settings)


In [ ]:
json_list = []

for idx, row in content_df.iterrows():
            dic = {}
            dic['_index'] = indexname
            dic['_type'] = index_type
            source = {}
            source[itemid] = row[itemid]
            source['longtext'] = row['clean']
            dic['_source'] = source
            json_list.append(dic)

In [ ]:
# For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
try:
    res = helpers.bulk(es, json_list)
    print("successfully imported to elasticsearch.")
except Exception as e:
    print(f"error: {e}")

/usr/local/lib/python3.8/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


successfully imported to elasticsearch.


In [ ]:
search_query="ক্রিকেটার থেকে গাইড"

In [ ]:
# Complicated query
query = es.search(
    index=indexname,
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"longtext":search_query}}
                ],
                # "should":[
                #         {"match":{"":""}}
                # ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.title,_source.longtext
0,new_index1,_doc,t9Zeh4YBbS-hUYFRXEE2,18.160336,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অ...
1,new_index1,_doc,z9Zeh4YBbS-hUYFRbFX7,12.478073,"তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমতা...","তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমত..."
2,new_index1,_doc,TtZeh4YBbS-hUYFRdGd1,12.040806,আজ হাইকোর্টে টেট মামলার রায় ঘোষণা,আজ হাইকোর্টে টেট মামলার রায় ঘোষণা আজ হাইকোর্...
3,new_index1,_doc,ddZeh4YBbS-hUYFRcmEA,11.558689,হুগলীতে তৃণমূলের 'মাথাব্যাথা' গোষ্ঠী দ্বন্দ্ব...,হুগলীতে তৃণমূলের 'মাথাব্যাথা' গোষ্ঠী দ্বন্দ্...
4,new_index1,_doc,E9Zeh4YBbS-hUYFRdGh1,10.432931,তালিবানের আতঙ্ক দূর হয়ে ক্রিকেট এখন কবুল,তালিবানের আতঙ্ক দূর হয়ে ক্রিকেট এখন কবুল ইডে...
5,new_index1,_doc,39Zeh4YBbS-hUYFRc2TH,8.322492,সা রে গা মা পা মঞ্চে নজিরবিহীন সিদ্ধান্ত! জিত ...,সা রে গা মা পা মঞ্চে নজিরবিহীন সিদ্ধান্ত! জিত...
6,new_index1,_doc,TdZeh4YBbS-hUYFRZkqr,7.207029,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্র...,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্...
7,new_index1,_doc,6dZeh4YBbS-hUYFRZ0zW,7.189875,কী কাণ্ড! ক্রিকেটার থেকে খুনি হয়ে গেলেন মহম্মদ...,কী কাণ্ড! ক্রিকেটার থেকে খুনি হয়ে গেলেন মহম্ম...
8,new_index1,_doc,G9Zeh4YBbS-hUYFRblqP,7.064561,ক্রিকেট ছেড়ে পাকিস্তানের সেনাবাহিনীতে কাজ খুঁ...,ক্রিকেট ছেড়ে পাকিস্তানের সেনাবাহিনীতে কাজ খু...
9,new_index1,_doc,DdZeh4YBbS-hUYFRbFb7,7.046682,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই আইসিস...
